# OHIE

The Oregon Health Insurance Experiment (OHIE) was a program run in
Oregon, USA in 2008 in which certain residents of that state were
offered the opportunity to enroll in a subsidized health insurance
program.  To allocate this opportunity fairly, interested people
were invited to participate in a lottery.  The people who won the
lottery were then given the opportunity to apply to the insurance
program.  A subset of these people actually applied to the program,
and finally a subset of these applicants who were confirmed to be
eligible were granted insurance.

Since the opportunity to apply for insurance was allocated randomly,
this program is essentially a randomized experiment (although the
randomization was employed for fairness, not to facilitate
research).  In particular, there was great interest in the outcomes
over the subsequent several years of the people who were awarded
insurance, compared to those who participated in the lottery but
were not selected.  Since this selection was random, in principal
there can be no confounders that would distort comparisons between
selected and non-selected individuals.

In this notebook, we only consider the "baseline" information,
namely, characteristics of the individuals who applied to the
lottery.  We also know who "won" the lottery, who among those given
the opportunity to apply for insurance actually did so, and who
among those who applied for insurance were deemed to be eligible and
granted insurance.

A primary focus of this notebook is to use the OHIE data to
illustrate concepts from probability, including conditional
probabilities and conditional independence.

In [ ]:
import os
import pandas as pd
import numpy as np

Modify this string according to your section number (001 or 002):

In [ ]:
f = "stats206s002f21"

Now we load the OHIE data from a file.

In [ ]:
base = "/scratch/%s_class_root/%s_class/shared_data/datasets" % (f, f)
df = pd.read_csv(os.path.join(base, "oregonhie.csv.gz"))

Let's begin by looking at how many people fall into each category of
program participation.  There are five such categories: not
selected, selected but did not apply, applied but deemed ineligible,
applied and deemed eligible, and unknown.  The number of people in
each of these categories is given by the following code:

In [ ]:
status = df.groupby(["treatment", "applied_app", "approved_app"], dropna=False).size()
status

Note that above we use 'dropna=False', overriding the default, since
people who did not win the lottery have missing values for variables
that are only defined for lottery winners.

It is desirable to see these counts presented as proportions.  Since
'ds' above is a series, we first convert it to a dataframe, so we
can add a second column containing the proportions.

In [ ]:
status = pd.DataFrame(status)
status.columns = ["Count"]
status["Prop"] = status["Count"] / status["Count"].sum()
status

Below we check the overall dataset size, and confirm that the
proportions sum to 1, as they must do.

In [ ]:
print(status["Count"].sum())
print(status["Prop"].sum())

The 'proportions' calculated above can be viewed as estimates (based
on the sample) of the population probabilities for the five types of
program participation.  We have a random sample of approximately
75,000 from this hypothetical population, so the sample proportions
are very likely to be close to the population probabilities.  Below
we will sometimes refer to these "proportions" as "probabilities".
In some other situations, it is important to distinguish between
"sample probabilities" a synonym for "proportions", and "population
probabilities".

# Marginal probabilities of events and complements

The probability of an event is the sum of probabilities for all
outcomes belonging to the event.  Suppose we want to estimate the
probability of being selected, regardless of whether the person
subsequently applied for or was granted insurance.  This is the sum
of rows 1, 2, 3, and 4 (counting from 0) in 'ds'.  We can obtain
this estimated probability as follows:

In [ ]:
status.iloc[1:, :]["Prop"].sum()

The 'complementary event' to being selected is to not be selected.
That is, everyone involved is either selected or not selected, and
nobody can be both.  Therefore, another way to calculate the
probability of being selected is to take 1 minus the probability of
not being selected:

In [ ]:
1 - status.iloc[0, :]["Prop"]

The state of Oregon had a fixed budget for this program, and could
not know in advance how many of the people who were selected in the
lottery would be deemed eligible for insurance.  Therefore, the
probability of being selected in the lottery was presumably set to a
conservative level.

# Joint probabilities

Next we will illustrate joint probabilities by looking at the
proportions of people with different demographic characteristics who
were selected into the program.  These proportions should reflect
the fact that selection into the program was random.

We will do this using an age group variable that we construct below.
First we obtain the age of each subject in the first year of the
program, then ask Pandas to group the subjects into three groups of
equal size based on age.

In [ ]:
df["age"] = 2008 - df["birthyear_list"]
df["ageg"] = pd.qcut(df["age"], 3)

Next we construct a 'contingency table' showing how many people have
each combination of the age group and treatment variables.

In [ ]:
counts = df.groupby(["ageg", "treatment"]).size().unstack()
counts

The next line of code converts the counts above to proportions.
Note that we must sum twice since the first sum only sums over the
rows.

In [ ]:
probs = counts / counts.sum().sum()
probs

The previous cell contains sample proportions, which estimate joint
probabilities in the population. Again we can check that the sample
proportions sum to exactly 1.

In [ ]:
probs.sum().sum()

# Conditional probabilities of treatment assignment

Recall that a conditional probability is a joint probability divided
by a marginal probability.  In general P(A|B) = P(A,B) / P(B), where
A and B are two 'events'.  Below we will estimate the conditional
probabilities of being assigned (selected) to the treatment group
(i.e. winning the lottery) given a person's age group:

P(win lottery | age group) = P(win lottery, age group) / P(age group)

The only other possible outcome of the lottery is not winning, and
the probability of this happening is 1 minus the probability of
winning the lottery.  The conditional distribution of 'win lottery'
given 'age group' is the collection consisting of all probabilities
of either winning or not winning the lottery, for all of the age
groups.  That is, 6 separate ways to apply the above formula, since
we can set 'win lottery' to either false or true, and we can set
'age group' to any of the three age groups.

To estimate the conditional probabilities by age group, we first
calculate the marginal probabilities by age group, as follows.

In [ ]:
age_marg = probs.sum(1)
age_marg

In this case, these probabilities are approximately equal, since we
constructed the age groups using 'qcut' to have this property.
However, age is only measured to the nearest years and there are lot
of people with the same age.  So it is not possible to perfectly
divide the sample into thirds based on age.  Also, recall that given
a dataframe 'x', 'x.sum(1)' sums within the rows and 'x.sum(0)' sums
within the columns.

Next we construct the conditional probabilities, by dividing the
joint probabilities by the marginal probabilities. We cannot simply
use 'dp / mpa' since this would be dividing a dataframe by a series,
and Pandas doesn't know how to align structures with different
shapes.  The 'div' method divides a dataframe by a series, and
contains an additional argument so that 'x.div(y, 0)' means that
every column of 'x' is divided by 'y', and 'x.div(y, 1)' means that
every row of 'x' is divided by 'y'.  Note that in the first case,
the length of 'y' must be equal to the number of rows of 'x', and in
the second case the length of 'y' must be equal to the number of
columns of 'x'.

In [ ]:
age_cond = probs.div(age_marg, 0)
age_cond

To confirm that we have valid conditional probabilities, check that
the elements of 'dr' sum to 1.

In [ ]:
age_cond.sum(1)

The results in 'dr' are consistent with random assignment.  Within
each age band, the probability of being selected is almost exactly
0.4.  The fact that this proportion is nearly the same across all
age bands suggests that there was no inadvertent non-random
assignment by age.

We can also look at the conditional distributions by column,
calculated below.

In [ ]:
app_cond = probs.div(probs.sum(0), 1)

To confirm that this is a valid distribution, check that the
proportions sum to 1:

In [ ]:
app_cond.sum()

The fact that the conditional probabilities given treatment
assignment are roughly constant (at 1/3 per age band) is a
consequence of the random treatment assignment, and of the fact that
we defined the age bands to include equal fractions of the sample.

Conditional probabilities of applying for insurance

Now let's consider a non-randomized variable -- whether a person who
is randomly selected to be given the opportunity to apply for
insurance actually submits the application.

First we select the people who were selected (i.e. who won the
lottery) since those who are not selected cannot apply for insurance
under this program.

In [ ]:
dx = df.loc[df["treatment"] == "Selected", :]

Next we calculate counts and proportions for each combination of age
and 'applied_app', which tells us whether each person submitted the
application to obtain insurance.

In [ ]:
counts = dx.groupby(["ageg", "applied_app"]).size().unstack()
probs = counts / counts.sum().sum()
probs

The conditional probabilities for age groups ('dr') and
'applied_app' levels (dc) are computed as above.

In [ ]:
age_marg = probs.div(probs.sum(1), 0)
app_marg = probs.div(probs.sum(0), 1)

The values in 'dr' are of special interest, because they indicate
how demographic differences relate to an individual's decision to
apply to the program.

In [ ]:
age_marg

Note that the conditional probability of applying to the program is
greater for older people.

# Independence

Two events A and B are statistically independent if P(A, B) =
P(A)*P(B).  To illustrate this concept using the OHIE data, we
consider whether the event of applying for insurance is independent
of age, considering only people who were given the opportunity to
apply (i.e. who won the lottery).

We start by calculating marginal probabilities, as above.

In [ ]:
age_marg = probs.sum(1)
app_marg = probs.sum(0)

If two random variables are independent, then the joint
probabilities can be constructed as the product of the marginal
probabilities.  We can obtain these as follows:

In [ ]:
ind = np.outer(age_marg, app_marg)
ind

These joint probabilities represent the closest exactly independent
distribution to the observed distribution of our data.

Next we can compare these exactly independent joint probabilities to
the observed joint probabilities.  These differences should be
exactly zero in the population if the random variables are
independent.  Here we are working with a sample not a population, so
we do not expect to get exact zeros.

In [ ]:
resid = probs - ind
resid

The differences above are almost 3 percentage points.  Later in the
course we will learn to more formally assess whether such are large
enough to suggest that the random variables in the underlying
population are statistically dependent.  Also note that the
differences above are a form of residual and therefore sum to zero.

In [ ]:
resid.sum().sum()

# Conditional independence

Finally we consider the possible dependence between age group and
'applied_app', already discussed above, but turn now to the question
of whether their dependence changes when conditioning on a third
variable, 'zip_msa_list'.  This variable indicates whether the
individual lives in a rural or urban area.

In [ ]:
urban = dx.loc[dx["zip_msa_list"] == "Zip code of residence in a MSA", :]
rural = dx.loc[dx["zip_msa_list"] == "Zip code of residence NOT in a MSA", :]

Below we calculate the closest exactly independent joint
distribution to the observed joint distribution for rural people.

In [ ]:
rural_counts = rural.groupby(["ageg", "applied_app"]).size().unstack()
rural_probs = rural_counts / rural_counts.sum().sum()
rural_ind = np.outer(rural_probs.sum(1), rural_probs.sum(0))

Now we calculate the closest exactly independent joint distribution
to the observed joint distribution for urban people.

In [ ]:
urban_counts = urban.groupby(["ageg", "applied_app"]).size().unstack()
urban_probs = urban_counts / urban_counts.sum().sum()
urban_ind = np.outer(urban_probs.sum(1), urban_probs.sum(0))

Next we form residuals.

In [ ]:
rural_resid = rural_probs - rural_ind
urban_resid = urban_probs - urban_ind

At this point we can only inspect these residuals, which are
specific to either rural or urban people, and compare them to
residuals for all people calculated above.  If the stratified
residuals (for urban or rural people) tended to be much smaller than
the unstratified residuals, then we might conclude that any
dependence between age and whether a person applies for insurance
('applied_app') is removed by conditioning on rurality
('zip_msa_list'). In this case, this does not seem to hold.  Later
in the course we will consider more formal ways for making this
assessment.  At this point, we must rely on our subjective
judgment.